In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json
import sys 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
# import stemmer as hindi_stemmer


In [ ]:
def tr_flatten(d,lc):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
        'contextual_label':lc[d['tweet_id']]
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] +' '+i['tweet'], #flattening comments(appending one after the other)
                'contextual_label':lc[i['tweet_id']]
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] +' '+ i['tweet'] +' '+ j['tweet'], #flattening replies
                        'contextual_label':lc[j['tweet_id']]
                    })
    return flat_text

In [ ]:
def te_flatten(d):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] + i['tweet'],
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] + i['tweet'] + j['tweet'],
                    })
    return flat_text

In [ ]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet, english_stemmer, stopword):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
    tokens = []
    for token in tweet.split():
        if token not in stopword:
            token = english_stemmer.stem(token)
            token = hi_stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [ ]:
import re
words_dict  = { "तैराक":"तैर",
                "चालाक":"चाल",
                "कूलाक":"कूल",
                "बेलन":"बेल",
                "मिलाप":"मिल",
                "चुपचाप": "चुप",
                "निकास":"निकस",
                "लुकास":"लुक",
                }

suffixes = {
	    1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],  
            2: ["तृ","ान","ैत","ने","ाऊ","ाव","कर", "ाओ", "िए", "ाई", "ाए", "नी", "ना", "ते", "ीं", "ती",
                "ता", "ाँ", "ां", "ों", "ें","ीय", "ति","या", "पन", "पा","ित","ीन","लु","यत","वट","लू"],     
            3: ["ेरा","त्व","नीय","ौनी","ौवल","ौती","ौता","ापा","वास","हास","काल","पान","न्त","ौना","सार","पोश","नाक",
                "ियल","ैया", "ौटी","ावा","ाहट","िया","हार", "ाकर", "ाइए", "ाईं", "ाया", "ेगी", "वान", "बीन",
                "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं","कला","िमा","कार",
                "गार", "दान","खोर"],     
            4: ["ावास","कलाप","हारा","तव्य","वैया", "वाला", "ाएगी", "ाएगा", "ाओगी", "ाओगे", 
                "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां",
                "त्वा","तव्य","कल्प","िष्ठ","जादा","क्कड़"],     
            5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां", "अक्कड़","तव्य:","निष्ठ"],
}

special_suffixes = ["र्", "ज्य","त्य"]
dict_special_suffixes = {"र्":"ृ",
                         "ज्य":"ज्",
                         "त्य":"त्"}

def hi_stem(word, clean=False,chars=None):
    if clean == True:
        word = clean_text(word, chars)
    
    ans = word
    bl = False
    
    if word in words_dict.keys():
        return words_dict[word]
    
    for L in 5, 4, 3, 2, 1:
        if len(word) > L + 1:
            for suf in suffixes[L]:
                if word.endswith(suf):
                    ans = word[:-L]
                    bl =True
        if bl == True:
            break
                    
    if bl == True:
        for suf in suffixes[1]:
            if ans.endswith(suf):
                # use case - गानेवाला
                ans = hi_stem(ans)
  
    for suf in special_suffixes:
        if ans.endswith(suf):
            l = len(suf)
            ans = ans[:-l]
            ans += dict_special_suffixes[suf]

    return ans

def clean_text(text, chars=None):
    if chars == None:        
        text = re.sub(r"[()\"#/@;:<>{}`+=~|!?,']", "", text)
    else:
        text = re.sub(r"[" +chars+ "()\"#/@;:<>{}`+=~|!?,']", "", text)
    return text

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
english_stopwords = stopwords.words("english")

with open('/content/drive/MyDrive/HASOC_files/final_stopwords.txt', encoding = 'utf-8') as f:
  hindi_stopwords = f.readlines()
  for i in range(len(hindi_stopwords)):
      hindi_stopwords[i] = re.sub('\n','',hindi_stopwords[i])

stopword = english_stopwords + hindi_stopwords
english_stemmer = SnowballStemmer("english")

In [ ]:
base_addreess = "/content/drive/MyDrive/HASOC/Train/contextual_2022_train (1)"
directories = []
for i in glob(base_addreess+"/Train/Hinglish/*/"):
  for j in glob(i+'*/'):
      directories.append(j)
data = []
for i in directories:
  with open(i+'data.json', encoding='utf-8') as f:
      data.append(json.load(f))

contextual_labels = []
for i in directories:
  with open(i+'contextual_labels.json', encoding='utf-8') as f:
      contextual_labels.append(json.load(f))

data_label = []
for i in range(len(contextual_labels)):
  for j in tr_flatten(data[i], contextual_labels[i]):
      data_label.append(j)




In [ ]:
# test

base_addreess = "/content/drive/MyDrive/hasoc_test/test_data"
directories = []
for i in glob(base_addreess+"/Train/Hinglish/*/"):
  for j in glob(i+'*/'):
      directories.append(j)
data = []
for i in directories:
  with open(i+'data.json', encoding='utf-8') as f:
      data.append(json.load(f))

# contextual_labels = []
# for i in directories:
#   with open(i+'contextual_labels.json', encoding='utf-8') as f:
#       contextual_labels.append(json.load(f))

data_test = []
for i in range(len(data)):
  for j in te_flatten(data[i]):
      data_test.append(j)

In [ ]:
len(data_test)

996

In [ ]:
data_label

In [ ]:
train_len = len(data_label)
df = pd.DataFrame(data_label, columns = data_label[0].keys(), index = None)
print("Multiclass Distribution")
print(df['contextual_label'].value_counts())





Multiclass Distribution
NONE    2390
SHOF    1636
CHOF     888
Name: contextual_label, dtype: int64


In [ ]:
train_len = len(data_label)
df_test = pd.DataFrame(data_test, columns = data_test[0].keys(), index = None)
print("Multiclass Distribution")
# print(df['contextual_label'].value_counts())

Multiclass Distribution


In [ ]:
df_test

,tweet_id,text
0,1445930336039358469,"Last warning to Endia, stop sponsoring #earthq..."
1,1445931080071069703,"Last warning to Endia, stop sponsoring #earthq..."
2,1445969052451573763,"Last warning to Endia, stop sponsoring #earthq..."
3,1445933724726489089,"Last warning to Endia, stop sponsoring #earthq..."
4,1446062615621373954,"Last warning to Endia, stop sponsoring #earthq..."
...,...,...
991,1467751712978989056,"On December 6, 1992, a crowd of almost 150,000..."
992,1467810485701472256,"On December 6, 1992, a crowd of almost 150,000..."
993,1467829444563779590,"On December 6, 1992, a crowd of almost 150,000..."
994,1467858643798216705,"On December 6, 1992, a crowd of almost 150,000..."


In [ ]:
df.contextual_label.unique()

array(['SHOF', 'CHOF', 'NONE'], dtype=object)

In [ ]:
d = {'SHOF' : 0  , 'CHOF' : 1 , 'NONE' : 2}

In [ ]:
df.contextual_label = df.contextual_label.map(d)

In [ ]:
tweets = df.text
multi_class_y = df.contextual_label 

cleaned_tweets = [clean_tweet(tweet, english_stemmer, stopword) for tweet in tweets]

In [ ]:
#applying class weight
from sklearn.utils import compute_class_weight
classweight = dict(zip([0,1,2],compute_class_weight(class_weight = 'balanced',y=df['contextual_label'].values,classes=[0,1,2])))
classweight

{0: 1.0012224938875305, 1: 1.8445945945945945, 2: 0.6853556485355649}

In [ ]:
tweets_test = df_test.text

In [ ]:
cleaned_tweets_test = [clean_tweet(tweet, english_stemmer, stopword) for tweet in tweets_test]

In [ ]:
len(cleaned_tweets_test)

996

In [ ]:
vectorizer = TfidfVectorizer(min_df = 5)

X = vectorizer.fit_transform(cleaned_tweets)
X = X.todense()



In [ ]:
X_test = vectorizer.transform(cleaned_tweets_test)

In [ ]:
X_test = X_test.todense()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, multi_class_y, test_size=0.2, random_state=42)
classifier = KNeighborsClassifier(5)
classifier.fit(X_train, y_train)



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


KNeighborsClassifier()

In [ ]:
y_pred = classifier.predict(X_val)
print("With K-nearest Neighbour:")
print(classification_report(y_val, y_pred))



With K-nearest Neighbour:
              precision    recall  f1-score   support

           0       0.49      0.33      0.39       333
           1       0.58      0.51      0.54       174
           2       0.61      0.78      0.68       476

    accuracy                           0.58       983
   macro avg       0.56      0.54      0.54       983
weighted avg       0.56      0.58      0.56       983



LogisticRegression 

In [ ]:
from sklearn import linear_model

In [ ]:
reg = model = LogisticRegression(C = 5,penalty ='l2'  , solver = 'newton-cg' , random_state = 42)

In [ ]:
reg.fit(X, multi_class_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


LogisticRegression(C=5, random_state=42, solver='newton-cg')

In [ ]:
y_pred = reg.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
len(X_test)

996

In [ ]:
df_submit = df_test

In [ ]:
df_submit

,tweet_id,label
0,1445930336039358469,NONE
1,1445931080071069703,NONE
2,1445969052451573763,NONE
3,1445933724726489089,NONE
4,1446062615621373954,NONE
...,...,...
991,1467751712978989056,NONE
992,1467810485701472256,SHOF
993,1467829444563779590,SHOF
994,1467858643798216705,SHOF


In [ ]:
df_submit = df_test.drop(['text'], axis=1)

In [ ]:
df_submit['label'] = y_pred

In [ ]:
d = { 0 : 'SHOF'  ,   1 : 'CHOF',   2 : 'NONE'}

In [ ]:
df_submit.label = df_submit.label.map(d)

In [ ]:
df_submit.to_csv('task2.csv',index = False)

In [ ]:
y_pred = reg.predict(X_val)
print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

With K-Nearest Neighbour:
              precision    recall  f1-score   support

           0       0.51      0.48      0.49       333
           1       0.65      0.53      0.59       174
           2       0.67      0.75      0.71       476

    accuracy                           0.62       983
   macro avg       0.61      0.59      0.60       983
weighted avg       0.62      0.62      0.62       983



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
# f = open("result/lg_agn.txt",'w')
# f.write(f"LogisticRegression(C = {0},tol = {0},penalty ={0})")
# f.close()
from sklearn.metrics import f1_score
C = [1,5,25,50,100,200]
penalty = ["l2" , 'l1' ,'elasticnet']
solver = ['newton-cg', 'lbfgs', 'sag','liblinear','saga']
# l1_ratio = [0.0,0.3,0.5,0.8,1.0]

In [ ]:
score = 0
for i in C:
  for k in penalty:
    for l in solver :
      print(i,k,l)
      try:
        model = LogisticRegression(C = i,penalty =k  , solver = l , random_state = 42)
        model.fit(X_train,y_train)
      except :
        continue
      pred_m = model.predict(X_val)
      f = f1_score(y_val,pred_m,average = 'macro')
      if(score<f):
          score = f
          # f = open("result/lg_agn.txt",'w')
          # f.write(f"LogisticRegression(C = {i},penalty = {k},solver={l})")
          # f.close()
          # joblib.dump(model,"result/agn_lg.joblib")
          print(i,k,l,score)
          # print(score)
              

1 l2 newton-cg


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


1 l2 newton-cg 0.5892821466587126
1 l2 lbfgs


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1

1 l2 sag


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


200 l1 newton-cg
200 l1 lbfgs
200 l1 sag
200 l1 liblinear


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


200 l1 saga
200 elasticnet newton-cg
200 elasticnet lbfgs
200 elasticnet sag
200 elasticnet liblinear
200 elasticnet saga


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
model3 = SVC(C = 1 ,kernel= 'linear')
model3.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


SVC(C=1, kernel='linear')

In [ ]:
y_pred = model3.predict(X_val)
print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


With K-Nearest Neighbour:
              precision    recall  f1-score   support

           0       0.51      0.47      0.49       333
           1       0.73      0.52      0.61       174
           2       0.65      0.76      0.70       476

    accuracy                           0.62       983
   macro avg       0.63      0.58      0.60       983
weighted avg       0.62      0.62      0.61       983



In [ ]:
mx = 0
for kernel in ['linear', 'poly', 'rbf']:
  for C in [1,5,10,25,50,100,200]:
    if(kernel=='poly'):
      if(C != 200):
        for degree in [1,2,3,4]:
            model = SVC(C = C,kernel=kernel,degree=degree)
            model.fit(X_train,y_train)
            pred = model.predict(X_val)
            f = f1_score(y_val,pred,average='macro')
            if(mx<f):
                mx = max(mx,f)
                # s = open('result/gen_svm.txt' , 'w')
                # s.write(f"SVC(C= {C}, degree = {degree} , kernel = {kernel}) score={mx}")
                # s.close()
                # joblib.dump(model,'result/gen_svm.joblib')
                print(kernel,C,degree,mx)
            print(f,f"C = {C} , kernel = {kernel} , degree = {degree}")
    else:
        model = SVC(C = C,kernel=kernel)
        model.fit(X_train,y_train)
        pred = model.predict(X_val)
        f = f1_score(y_val,pred,average='macro')
        if(mx<f):
            mx = max(mx,f)
            # s = open('result/gen_svm.txt' , 'w')
            # s.write(f"SVC(C= {C}, degree = {1} , kernal = {kernel}) score={mx}")
            # s.close()
            # joblib.dump(model,'result/gen_svm.joblib')
            print(kernel,C,mx)
        print(f,f"C = {C} , kernel = {kernel}")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


linear 1 0.6005841200306409
0.6005841200306409 C = 1 , kernel = linear


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


0.6004888112558731 C = 5 , kernel = linear


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


...

0.5545613216164055 C = 50 , kernel = rbf


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


0.5534546005069955 C = 100 , kernel = rbf


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


0.5541178457685895 C = 200 , kernel = rbf


MNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB(alpha=0.06891)
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


MultinomialNB(alpha=0.06891)

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.49      0.49       333
           1       0.51      0.54      0.53       174
           2       0.68      0.66      0.67       476

    accuracy                           0.58       983
   macro avg       0.56      0.56      0.56       983
weighted avg       0.58      0.58      0.58       983



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
score =0
for i in np.arange(1e-05,1e-01,1e-04):
  print(i)
  model = MultinomialNB(alpha=i)
  model.fit(X_train,y_train)
  pred = model.predict(X_val)
  f = f1_score(y_val,pred,average='macro')
  if(score<f):
    score = f
    # f = open("result/gen_mnb.txt",'w')
    # f.write(f"MultinomialNB(alpha = {i})")
    # f.close()
    # joblib.dump(model,'result/gen_mnb.joblib')
    print(i,score)

1e-05
1e-05 0.5767259714042453
0.00011
0.00011 0.5772294685011076
0.00021


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn

0.00021 0.5806848523834162
0.00031000000000000005
0.00041000000000000005
0.00051
0.0006100000000000001


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn

0.00071
0.0008100000000000001
0.0009100000000000001


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn

0.09941
0.09951
0.09961


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn

0.09971000000000001
0.09981
0.09991


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn

In [ ]:
score

0.5806848523834162

In [ ]:
0.00021

In [ ]:
model = MultinomialNB(alpha=0.00021)
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


MultinomialNB(alpha=0.00021)

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.54      0.52       333
           1       0.58      0.52      0.55       174
           2       0.69      0.66      0.67       476

    accuracy                           0.60       983
   macro avg       0.59      0.58      0.58       983
weighted avg       0.60      0.60      0.60       983



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


randomforest

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
from sklearn.metrics import f1_score
max_features = ['auto', 'sqrt', 'log2']
max_depth = [4,5,6,7,8]
criterion =['gini', 'entropy'] 

In [ ]:
score=0
for i in max_features:
  for j in max_depth:
    for k in criterion:
      print(i,j,k)
      model=RandomForestClassifier(max_features=i,max_depth=j,criterion=k)
      model.fit(X_train,y_train)
      pred=model.predict(X_val)
      f=f1_score(y_val,pred,average='macro')
      if(score<f):
        score=f
        # f=open("result/gen_rf.txt",'w')
        # f.write(f"RandomForestClassifier(C={i},tol={j},penalty={k}) score={score}")
        # f.close()
        # joblib.dump(model,'result/gen_rf.joblib')
        print(i,j,k,score)

auto 4 gini


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


auto 4 gini 0.3754346311429257
auto 4 entropy


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


auto 5 gini


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


log2 8 gini


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


log2 8 entropy


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
model=RandomForestClassifier()
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


RandomForestClassifier()

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.44      0.45       333
           1       0.66      0.53      0.59       174
           2       0.65      0.73      0.69       476

    accuracy                           0.60       983
   macro avg       0.59      0.57      0.58       983
weighted avg       0.59      0.60      0.59       983



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


XGboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model=XGBClassifier()
model.fit(X_train,y_train)

XGBClassifier(objective='multi:softprob')

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.35      0.42       333
           1       0.73      0.38      0.50       174
           2       0.60      0.82      0.69       476

    accuracy                           0.58       983
   macro avg       0.61      0.52      0.53       983
weighted avg       0.59      0.58      0.56       983



In [ ]:
# g=open("result/gen_xgb.txt",'w')
# g.write(f"XGBClassifier(min_child_weight={0},gamma={0},subsample={0})")
# g.close()
from sklearn.metrics import f1_score
min_child_weight = [1, 5, 10]
gamma =[0.5, 1, 1.5, 2, 5]
subsample= [0.6, 0.8, 1.0]

In [ ]:
from xgboost import XGBClassifier 
score=0 
for i in min_child_weight: 
  for j in gamma: 
    for k in subsample: 
      print(i,j,k) 
      model=XGBClassifier(min_child_weight=i,gamma=j,subsample=k,class_weight=classweight) 
      model.fit(X_train,y_train) 
      pred_m=model.predict(X_val) 
      f=f1_score(y_val,pred_m,average='macro') 
      
      if(score<f): 
        score=f 
        print(i,j,k,score)

ADABoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
score=0
for i in [50,70,80,90,100,120]:
  for j in np.arange(1e-02,1,1e-01):
    print(i,j)
    model=AdaBoostClassifier(n_estimators=i,learning_rate=j)
    model.fit(X_train,y_train)
    pred_m=model.predict(X_val)
    f=f1_score(y_val,pred_m,average='macro')
    if(score<f):
      score=f
      # g=open("result/gen_adb.txt",'w')
      # g.write(f"AdaBoost(n_estimators={i},learning_rate={j})")
      # g.close()
      print(i,j,score)
      # joblib.dump(model,"result/gen_adb.joblib")

In [ ]:
model=AdaBoostClassifier(n_estimators=i,learning_rate=j)
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_val)
# print("With K-Nearest Neighbour:")
print(classification_report(y_val, y_pred))

ANN

In [ ]:
le = LabelEncoder() #label encoding labels for training Dense Neural Network
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

y_train = to_categorical(y_train,num_classes=3)
#y_val = to_categorical(y_val,num_classes=3)
print(y_train.shape)


(3931, 3)


In [ ]:
model = Sequential(
  [
      Dense(64, activation="relu"),
      Dense(32, activation="relu"),
      Dense(3, activation="softmax"),
  ]
)

model.compile('adam', loss='categorical_crossentropy', metrics = ['accuracy']) #compiling a neural network with 3 layers for classification
model.fit(X_train, y_train, epochs = 5, batch_size = 32)



Epoch 1/5
123/123 [==============================] - 1s 2ms/step - loss: 0.9337 - accuracy: 0.5480
Epoch 2/5
123/123 [==============================] - 0s 2ms/step - loss: 0.7276 - accuracy: 0.6813
Epoch 3/5
123/123 [==============================] - 0s 2ms/step - loss: 0.6086 - accuracy: 0.7393
Epoch 4/5
123/123 [==============================] - 0s 2ms/step - loss: 0.5076 - accuracy: 0.7891
Epoch 5/5
123/123 [==============================] - 0s 2ms/step - loss: 0.4402 - accuracy: 0.8186


In [ ]:
y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis = 1)
y_pred = y_pred.reshape(len(y_pred))    

print("With MLP:")
print(classification_report(y_val, y_pred)) 

With MLP:
              precision    recall  f1-score   support

           0       0.61      0.56      0.58       174
           1       0.66      0.74      0.70       476
           2       0.50      0.43      0.46       333

    accuracy                           0.61       983
   macro avg       0.59      0.58      0.58       983
weighted avg       0.60      0.61      0.60       983

